In [22]:
import pandas as pd
import numpy as np
import nltk as nltk
import matplotlib.pyplot as plt
%matplotlib inline

import zipfile
import pickle
from os.path import join as path, dirname

try:
    from IPython.core.display import HTML

    def pprint(df):
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(HTML(pd.DataFrame(df).to_html()))
except:
    def pprint(df):
        print(df)

from sklearn.model_selection import train_test_split

In [23]:
print('Training data:')
df_train = pd.read_csv(path('data', 'train.csv'), index_col='PassengerId')
print(df_train.shape)
pprint(df_train.head())

print('Test data:')
df_test = pd.read_csv(path('data', 'test.csv'), index_col='PassengerId')
print(df_test.shape)
pprint(df_test.head())

Training data:
(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Test data:
(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [40]:
def count_passengers(df):
    return df.shape[0]

def get_states(df, colName):
    return df[colName].unique()

def count_states(df, colName):
    return df[colName].value_counts()

def count_survived_column(df, colName):
    return df[["Survived", colName]].groupby(colName).agg(np.sum)

def count_survived(df):
    return count_states(df, "Survived")[1]

def get_surviveds(df):
    return df.loc[df["Survived"]==1]

In [35]:
def get_counts(df, colExcept):
    counts = []
    for col in df.columns:
        if(col not in colExcept):
            newDF = count_survived_column(df, col)
            newDF.name = col
            counts.append(newDF)
    return counts

In [37]:
def prepare_df(df):
    df_copy = df.copy()
    df_copy["Age"].fillna(-1, inplace=True)
    return df_copy
df_train = prepare_df(df_train)

In [38]:
def get_probability(r, df):
    c = get_counts(df, ["Survived", "Name", "Ticket", "Cabin", "Fare"])
    n_surv = count_survived(df)
    prob = (n_surv+1.)/(count_passengers(df)+len(get_states(df, "Survived")))
    for col, value in r.iteritems():
        for i in c:
            if(i.name == col):
                try:
                    prob *= (i["Survived"].loc[value]+1.)/(n_surv+i.shape[0])
                except:
                    if(np.isnan(value)):
                        value=-1                    
                        prob *= (i["Survived"].loc[value]+1.)/(n_surv+i.shape[0])
                    else:
                        prob *= 1./(n_surv+i.shape[0])
                break
    return prob

In [28]:
def invert_survived(df):
    df_copy = df.copy()
    df_copy.loc[(df_copy["Survived"]==0), "Survived"]=2
    df_copy.loc[(df_copy["Survived"]==1), "Survived"]=0
    df_copy.loc[(df_copy["Survived"]==2), "Survived"]=1
    return df_copy

In [41]:
X_full = df_train.copy()
X_full.drop(['Cabin', "Fare", "Ticket", "Name"], axis=1, inplace=True)
print(X_full)
y = df_train["Survived"]
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)
X_valid_full.drop(['Survived'], axis=1, inplace=True)
X_train_full_inv = invert_survived(X_train_full)
results = [0 if get_probability(r, X_train_full)<get_probability(r, X_train_full_inv) else 1 for index, r in X_valid_full.iterrows()]   
results=pd.Series(results)
results.index.name = "PassengerId"
results.index = X_valid_full.index
results.name="Survived"
print(results)
encerts = 0
total = 0
for item, value in results.iteritems():
    if(value == y_valid.loc[item]):
            encerts+=1
    total+=1.
print(encerts/total)

             Survived  Pclass     Sex   Age  SibSp  Parch Embarked
PassengerId                                                       
1                   0       3    male  22.0      1      0        S
2                   1       1  female  38.0      1      0        C
3                   1       3  female  26.0      0      0        S
4                   1       1  female  35.0      1      0        S
5                   0       3    male  35.0      0      0        S
...               ...     ...     ...   ...    ...    ...      ...
887                 0       2    male  27.0      0      0        S
888                 1       1  female  19.0      0      0        S
889                 0       3  female  -1.0      1      2        S
890                 1       1    male  26.0      0      0        C
891                 0       3    male  32.0      0      0        Q

[891 rows x 7 columns]


C:\Users\usuario\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


PassengerId
496    0
649    0
279    0
32     1
256    1
      ..
781    1
838    0
216    1
834    0
373    0
Name: Survived, Length: 179, dtype: int64
0.8379888268156425


In [43]:
df_train_inv = invert_survived(df_train)
results = [0 if get_probability(r, df_train)<get_probability(r, df_train_inv) else 1 for index, r in df_test.iterrows()]   
results=pd.Series(results)
results.index.name = "PassengerId"
results.index = df_test.index
results.name="Survived"
results.to_frame().to_csv("submission.csv")